# Apprentissage d'ensemble, et Forrêt aléatoire

 Extrait du livre d'Aurélien Géron

## Setup

### Les imports

In [19]:
%reset -f
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn.linear_model
import sklearn.svm
import sklearn.ensemble
import sklearn.datasets
import sklearn.tree
import sklearn.model_selection
import sklearn.metrics
import sklearn.neural_network



plt.style.use("default")


"data for the whole notebook"
X, y = sklearn.datasets.make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)



def plot_decision_boundary(ax,prediction_func, X, y, title="", extent=[-2, 3, -2, 2],alpha=0.5,plot_data=True):
    
    x1s = np.linspace(extent[0], extent[1], 100)
    x2s = np.linspace(extent[2], extent[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.stack([x1.reshape(-1), x2.reshape(-1)],axis=1)
    y_pred = prediction_func(X_new).reshape(x1.shape)
    
    ax.imshow(y_pred,extent=extent,origin="lower",interpolation="bilinear",cmap="jet",alpha=alpha)
            
    if plot_data:
      ax.scatter(X[:, 0],X[:, 1],marker=".",c=y,cmap="jet",linewidths=0)
      ax.set_title(title)
      ax.set_xlabel(r"$x_1$", fontsize=18)
      ax.set_ylabel(r"$x_2$", fontsize=18)



### Regarder les données

 On utilise le jeu `moon` de `sklearn`. Les instances appartiennent à deux catégories. Il y a deux descripteurs. 

 ***A vous:*** Pourquoi s'appelle-t-il moon ? 

In [20]:
fig,(ax0,ax1)=plt.subplots(1,2,figsize=(10,5),sharey=True)

def plot_one(ax,X,y,title):
  ax.scatter(
      X[:,0],
      X[:,1],
      c=y,
      edgecolor="w",
      cmap="jet"
  )
  ax.set_title(title)
  ax.set_aspect("equal")
  
  
plot_one(ax0,X_train,y_train,"train")
plot_one(ax1,X_test,y_test,"test")
print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)

## Classifieurs votants

Prendre une décision à plusieur c'est plus sur. C'est dû à "la sagesse de la foule".  En analyse de donnée, le fait d'utiliser plusieurs modèles est appelé "ensemble learning".


Vous pouvez entrainer de nombreux arbre de décisions sur des sous-parties aléatoire du jeu train, puis agréger leurs réponses en prenant la classe prédite majoritairement. C'est le principe de la forêt aléatoire (random-forest). Malgré sa simplicité, c'est l'un des algorithmes d'apprentissage automatique les plus puissants disponibles aujourd'hui. 

Les solutions gagnantes des concours d'apprentissage automatique (ex: l'historique compétition Netflix) utilise souvent l'Ensemble learning. 


***A vous:*** D'après vous: que fallait-il faire durant cette fameuse compétition Netflix ? 

### Vote dur (Hard voting)

On appelle ainsi le vote majoritaire. Si de nombreux classifier binaire ont une accuracy de 51%, le résultat de leur vote majoritaire aura bien meilleure accuracy.

Comment cela est-il possible ? L'analogie suivante peut aider à comprendre. Considérons une piece qui avec proba 51% donne pile.  
* En lançant 1000 fois cette piece, la probabilité d'obtenir une majorité de pile est de 75%. 
* En la lançant 10 000 fois, la probabilité grimpe à 97%.


***A vous:*** 
* Simuler l'expérience décrite ci-dessus. Vérifiez le coup des 75%/97%. 
* Donnez une expression mathématique (à base de coefficients binomiaux) pour calculer ces 75%/97%. Puis donnez une manière d'approximer cette approximation numériquement (souvenirs des cours de proba).
* Pour que ces calculs fonctionnent on a du faire une hypothèse fondammentale sur les différents lancés, laquelle ? 

### Différents types de classificateurs

Pour que les méthodes d'ensemble fonctionnent il faut que les modèles soient le plus indépendants possible. Une technique possible est  d'utiliser des algorithmes très différents.


In [21]:
log_clf = sklearn.linear_model.LogisticRegression(random_state=42)
rnd_clf = sklearn.ensemble.RandomForestClassifier(random_state=42)
svm_clf = sklearn.svm.SVC(random_state=42)

voting_clf = sklearn.ensemble.VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard');

In [22]:
voting_clf.fit(X_train, y_train);

In [23]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, sklearn.metrics.accuracy_score(y_test, y_pred))

Et voilà ! Le classement par votes est légèrement meilleurs que chacun des classements individuels.

### Vote doux (Soft voting) 

Si tous les classificateurs d'un ensemble sont capables d'estimer les probabilités, vous pouvez agréger leur prédictions en prenant la moyenne des probabilités. 
 C'est ce que l'on appelle le "soft voting". Il est souvent plus performant que le vote dur car il donne plus de poids aux classifieur très confiants en leurs prédictions.



In [24]:
log_clf = sklearn.linear_model.LogisticRegression(random_state=42)
rnd_clf = sklearn.ensemble.RandomForestClassifier(random_state=42)
svm_clf = sklearn.svm.SVC(probability=True,random_state=42) #observez la variante ici


voting_clf = sklearn.ensemble.VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train);

In [25]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, sklearn.metrics.accuracy_score(y_test, y_pred))

***A vous:*** Comment créer une méthode d'ensemble avec des modèles de regression ? 

## Bagging ensembles

### Intro

Une autre approche pour créer un ensemble de modèles assez indépendant, consiste à utiliser le même algorithme, mais  entrainé sur différents sous-échantillons aléatoires du jeu Train. 

* Lorsque l'échantillonnage est effectué avec replacement, cette méthode est appelée "bagging" (abréviation de "bootstrap aggregating"). 
* Lorsque l'échantillonnage est effectué sans replacement, cette méthode est appelée "pasting" (collage).

Chaque prédicteur individuel a un biais plus élevé que s'il était entrainé sur Train en entier, mais l'agrégation réduit un peu le biais et beaucoup la variance (ce qui est naturel n'est-ce pas?). 


### Codons


Ci-dessous, on entraine 500  arbres de décision, chacun avec 100 données Train tirée avec replacement (`bootstrap=True`). 

Le paramètre `n_jobs` indique à Scikit-Learn le nombre de  CPU à utiliser (-1 => tous les CPU disponibles). En effet, un tel algo est facile à paralélliser puisque chaque arbre se constuit indépendamment des autres.  

In [26]:
bag_clf = sklearn.ensemble.BaggingClassifier(
    sklearn.tree.DecisionTreeClassifier(random_state=42), 
    n_estimators=500,
    max_samples=100, #nombre d'instance tiré pour chaque arbre
    bootstrap=True, 
    n_jobs=-1, 
    random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [27]:
print("accuracy_score with:",sklearn.metrics.accuracy_score(y_test, y_pred))

In [28]:
tree_clf = sklearn.tree.DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print("accuracy_score without:",sklearn.metrics.accuracy_score(y_test, y_pred_tree))

***Note:*** Le `BaggingClassifier` de `sklearn` effectue automatiquement un soft-voting si le classificateur de base peut estimer les probabilités de classe (c'est-à-dire s'il a une méthode `predict_proba`), ce qui est le cas des `DecisionTreeClassifier`. 

In [29]:
fig,(ax0,ax1)=plt.subplots(1,2,figsize=(11,4),sharey=True)
plot_decision_boundary(ax0,tree_clf.predict, X, y,"Decision Tree")
plot_decision_boundary(ax1,bag_clf.predict, X, y,"Bagging")

Ci-dessus, vous pouvez comparer la frontière de décision d'un seul arbre de décision avec celle d'un ensemble de 500 arbres. Les prédictions de l'ensemble se généraliseront probablement beaucoup mieux.  

Ci-dessous on compare les proba prédites. Celles produites par l'ensemble sont beaucoup plus continues. 

In [30]:
fig,(ax0,ax1)=plt.subplots(1,2,figsize=(11,4))

def predic_func_tree(x):
  return tree_clf.predict_proba(x)[:,1]

def predic_func_bag(x):
  return bag_clf.predict_proba(x)[:,1]

"petit test"
#predic_func_tree([[1,2],[1,2],[1,2]])

plot_decision_boundary(ax0,predic_func_tree, X, y,"Decision Tree")
plot_decision_boundary(ax1,predic_func_bag, X, y,"Bagging")

### Utiliser le Bootstrap ou pas? 

Le bootstrap introduit un peu plus de diversité dans les sous-ensembles sur lesquels chaque modèles est entrainé. On obtient ainsi souvent une moindre variance. Mais si vous avez le temps, testez toujours `bagging` et `pasting` à l'aide d'une validation croisée par exemple. 

  

### Les instances hors-sac (out-of-bag)


Quand on tire aléatoirement des éléments de `Train` avec replacement (`bootstrap=True`), certains ne sont jamais tirés. On les appelle les 'out of bag' (oob). 


Notons $m$ le nombre d'instance (= la taille de Train). Par défaut, pour chacun des modèles entrainé, le `BaggingClassifier` échantillonne $m$ instances avec remplacement.  Un calcul classique montre que cela produit 37% d'instances oob. 
 Notons que ce ne sont pas les mêmes 37% d'un modèle à l'autre.

Pour chaque modèle les oob ne sont pas utilisé pour l'entrainement, on peut calculer un score de validation avec. C'est une sorte de validation croisée. 


***A vous:*** Faite des simulations  pour vérifier le chiffre des 63%. Plus précisément, on montre que quand $m$ devient grand, la proportion de donnée non piochée converge vers $ 1-e^{-1}$.

In [31]:
bag_clf = sklearn.ensemble.BaggingClassifier(
    sklearn.tree.DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, 
    max_samples=1.0,#le nombre d'instance tiré est égal à la taille de Train 
    bootstrap=True, 
    n_jobs=-1,
    random_state=42,
    oob_score=True
)

In [32]:
bag_clf.fit(X_train, y_train)
print("accuracy on oob data:",bag_clf.oob_score_)

 Vérifions que la validation donne un score proche de la prédiction sur le jeu `Test`


In [33]:
y_pred = bag_clf.predict(X_test)
print("accuracy on test data:",sklearn.metrics.accuracy_score(y_test, y_pred))

### Patches aléatoires et sous-espaces aléatoires


La classe `BaggingClassifier` permet également d'échantillonner les descripteurs (features). Deux hyperparamètres controle cet échantillonage: `max_features` et `bootstrap_features`. Ils fonctionnent de la même manière que les paramètres `max_samples` et `bootstrap`. Ainsi, chaque modèle sera entrainé sur un sous-ensemble aléatoire de descripteurs.

Ceci est particulièrement utile lorsque vous avez affaire à des entrées de grande dimension (comme des images). 

***Vocabulaire:***
* Quand on échantillonne les instances de Train, on parle de **Patches aléatoire**. 
* Quand  on garde tous les éléments de Train (`bootstrap=False` et `max_samples=1.0`) mais que l'on échantillone des descripteurs (`bootstrap_features=True` et/ou `max_features` < 1.0),  on parle de **Sous-espaces Aléatoires**.

La technique des sous-espaces aléatoires donnent une plus grande diversité de prédicteurs, ajoutant un peu plus de biais mais diminuant la variance.

## Forêt aléatoire (Random Forests)

### Définition


Une forêt aléatoire est un ensemble d'arbres de décision. Typiquement on utilise le bagging avec  `max_sample` fixés à la taille Train. 

Informatiquement: Au lieu de construire un `BaggingClassifier` et de lui passer un `DecisionTreeClassifier`, vous pouvez directement utiliser la classe `RandomForestClassifier`, qui est plus pratique et optimisée pour les arbres de décision ; de même, il existe une classe `RandomForestRegressor` pour les tâches de régression. 

Le code suivant entraîne un classificateur Random Forest avec 500 arbres (chacun limité à 16 nœuds au maximum), en utilisant tous les cœurs CPU disponibles 

In [34]:
rnd_clf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16, 
    n_jobs=-1, 
    random_state=42,
    )

rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)
print(y_pred_rf)

In [35]:
sklearn.ensemble.RandomForestClassifier?

À quelques exceptions près, un `RandomForestClassifier` possède tous les hyperparamètres d'un `DecisionTreeClassifier` (pour contrôler la façon dont les arbres sont construits), auxquels s'ajoute tous les hyperparamètres d'un `BaggingClassifier` pour contrôler la méthode d'ensemble.


Le BaggingClassifier suivant est à peu près équivalent au RandomForestClassifier précédent : 

In [36]:
bag_clf = sklearn.ensemble.BaggingClassifier(
        sklearn.tree.DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
        n_estimators=500, 
        max_samples=1.0, 
        bootstrap=True, 
        n_jobs=-1
    )

bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)


In [37]:
np.sum(y_pred == y_pred_rf) / len(y_pred)  # almost identical predictions

L'algorithme RandomForest introduit un caractère aléatoire supplémentaire par rapport au `BaggingClassifier` précédent. Lorsqu'on n'utilise pas tous les descripteur à la fois, leur tirage aléatoire est différent d'un noeud à l'autre des arbres. 


Il en résulte une plus grande diversité des arbres =>
* Biais plus grand
* Variance plus basse
* Meilleurs modèle en général.

Le paramètre qui contrôle ceci est `max_features`. Par défaut on a: 

    max_features= "auto" 
    
ce qui équivaut à :

    max_features=sqrt(nombre de features)

Attention: 

    max_feature = None


alors toutes les features sont utilisée (donc pas de tirage aléatoire). 

### Illustration graphique

Ci-dessous on a supperposer les frontière de décision de chacun des arbres, en mettant un paramètre de transparance très petit `alpha=0.05`. 

***A vous:*** Tracez maintenant la frontière de décision de la forêt. Cela devrait être proche de notre superposition. 

In [38]:
fig,ax=plt.subplots(1,1,figsize=(6, 4))

for i in range(15):
    tree_clf = sklearn.tree.DecisionTreeClassifier(max_leaf_nodes=16, random_state=42 + i)
    indices_with_replacement = np.random.randint(0, len(X_train), len(X_train))
    tree_clf.fit(X[indices_with_replacement], y[indices_with_replacement])
    plot_decision_boundary(ax,tree_clf.predict, X, y,  alpha=0.05, plot_data=(i==0))

### Importance des descripteurs

Dans un arbre de décision: Les descripteurs important apparaissent proche de la racine   (car les premières divisions sont celle qui séparent le mieux les données). 

Il est donc possible d'obtenir une estimation de l'importance d'un descripteur en calculant la profondeur moyenne à laquelle il apparaît sur tous les arbres de la forêt. Scikit-Learn calcule automatiquement cette profondeur pour chaque feature après l'entraînement et stocke le résultat dans l'attribut  `feature_importances_`. 

Par exemple, le code suivant entraîne un `RandomForestClassifier` sur le jeu iris qui a 4 descripteurs. La profondeur moyenne de chacun d'entre eux est donné en pourcentage (ils divisent par la hauteur totale de l'arbre). Plus le pourcentage est grand, plus le descripteur apparait en moyenne haut dans les arbres. 

In [39]:
iris = sklearn.datasets.load_iris()
rnd_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

### Extra-Tree

Il est possible de rendre les arbres encore plus aléatoires en utilisant des descripteurs aléatoire et également des seuils aléatoires (aucun processus d'optimisation). On appelle cela un `Extra-Tree`. A priori un tel arbre isolé n'a pas d'interêt. Mais une forêt d'extra-tree donne de bonnes prédictions.


## AdaBoost

### Intro 

Le Boosting fait référence à toute méthode d'Ensemble qui peut combiner plusieurs apprenants faibles pour en faire un apprenant fort. L'idée générale de la plupart des méthodes de boosting est d'entraîner les modèles de manière séquentielle, chacun essayant de corriger son prédécesseur. Il existe de nombreuses méthodes de boosting, mais les plus populaires son AdaBoost (= Adaptive Boosting) et Gradient Boosting. Commençons par AdaBoost. 

Une façon pour un nouveau modèle de corriger son prédécesseur est d'accorder un peu plus d'attention aux instances de `Train` que le prédécesseur prédit mal. Ainsi, les nouveaux modèles se concentrent de plus en plus sur les cas difficiles. C'est la technique utilisée par AdaBoost.


Détaillons cela. Le premier modèle utilise:
$$
 \sum_{Train} Loss(x_i,y_i) =  \sum_{Train} dist[model(x_i),y_i]
$$
On a ensuite assigner les poids $(r_i)$ aux instances: Celles mal classées auront un poids plus important. Et le second classifier sera entrainé avec une loss pondérée:
$$
 \sum_{Train} r_i \, Loss(x_i,y_i)
$$
On effectue une nouvelle prédiction avec le second classifier. On change les poids en fonction du résultat, et on recommence avec un troisième. etc.

In [40]:
ada_clf = sklearn.ensemble.AdaBoostClassifier(
    sklearn.tree.DecisionTreeClassifier(max_depth=1), 
    n_estimators=200,
    algorithm="SAMME.R", 
    learning_rate=0.5, 
    random_state=42)

ada_clf.fit(X_train, y_train);

In [41]:
fig,ax=plt.subplots()
plot_decision_boundary(ax,ada_clf.predict, X, y)

### Step by step

Implémentons à la main la méthode Adaboost. On utilise un classifier de type SVM (support vector machine) dont le principe est de trouver un hyperplan qui sépare au mieux les données. Mais les données sont préalablement augmentée par des "noyaux". Ici on utilise les noyaux "rbf" qui sont des gaussiennes ; voir [ici](https://en.wikipedia.org/wiki/Radial_basis_function_kernel)


Les autres hyperparamètres important de notre programme sont: 

* `C` est la constance de régularisation d'un SVM. C'est l'inverse de "alpha": plus `C` est grand, et moins le modèle est régulier. 
* `learning_rate`: vous devez pouvoir comprendre dans le programme comment il fonctionne. 



***A vous:*** Faites varier  les hyper-paramètre `C` et `learning_rate`. 
* Avec `C` trop petit, la frontière de décision sera trop tordue. 
* Avec `learning_rate` trop petit, on n'évolura pas asser d'un classifier à l'autre. Avec `learning_rate` trop grand ...




In [42]:
fig,axs=plt.subplots(4,2,figsize=(10,15))
axs=axs.reshape(-1)

m = len(X_train)
learning_rate=0.2
sample_weights = np.ones(m)


for i in range(len(axs)):
    model = sklearn.svm.SVC(kernel="rbf", C=0.03, random_state=42)
    #model = sklearn.tree.DecisionTreeClassifier(max_depth=4)
    model.fit(X_train, y_train, sample_weight=sample_weights)
    y_pred = model.predict(X_train)
    sample_weights[y_pred != y_train] *= (1 + learning_rate)
    plot_decision_boundary(axs[i],model.predict, X, y) 

***Astuce:*** Si votre ensemble AdaBoost sur-apprend, vous pouvez essayer de réduire le nombre d'itération, le learning rate ou bien régulariser plus fortement l'estimateur de base.

### inconvénient

Cette technique d'apprentissage séquentiel présente un inconvénient important : elle ne peut pas être parallélisée. Chaque prédicteur doit attendre que le précédent ai fini son entrainement.  

## Gradient Boosting $\hookleftarrow$

### Explications

Un autre algorithme de Boosting très populaire est le Gradient Boosting. Tout comme AdaBoost, le GradientBoost fonctionne en ajoutant séquentiellement des prédicteurs à un ensemble, chacun corrigeant son prédécesseur. Cependant, au lieu de modifier les poids des instances à chaque itération, on tente de diminuer les erreurs résiduelles du prédicteur précédent.
Voyons un exemple de régression simple utilisant les arbres de décision comme prédicteurs de base. C'est ce qu'on appelle le Gradient Tree Boosting, ou arbres de régression à gradient renforcé (GBRT). 


### Experimentation

In [43]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [44]:
tree_reg1 = sklearn.tree.DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y);

In [45]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = sklearn.tree.DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2);

In [46]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = sklearn.tree.DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3);

In [47]:
X_new = np.array([[0.8]])

In [48]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [49]:
y_pred

In [50]:
def plot_predictions(regressors, X, y, axes, label=None, style="r-", data_style="b.", data_label=None):
    x1 = np.linspace(axes[0], axes[1], 500)
    y_pred = sum(regressor.predict(x1.reshape(-1, 1)) for regressor in regressors)
    plt.plot(X[:, 0], y, data_style, label=data_label)
    plt.plot(x1, y_pred, style, linewidth=2, label=label)
    if label or data_label:
        plt.legend(loc="upper center", fontsize=16)
    plt.axis(axes)

plt.figure(figsize=(11,11))

plt.subplot(321)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h_1(x_1)$", style="g-", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Residuals and tree predictions", fontsize=16)

plt.subplot(322)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1)$", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Ensemble predictions", fontsize=16)

plt.subplot(323)
plot_predictions([tree_reg2], X, y2, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_2(x_1)$", style="g-", data_style="k+", data_label="Residuals")
plt.ylabel("$y - h_1(x_1)$", fontsize=16)

plt.subplot(324)
plot_predictions([tree_reg1, tree_reg2], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1)$")
plt.ylabel("$y$", fontsize=16, rotation=0)

plt.subplot(325)
plot_predictions([tree_reg3], X, y3, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_3(x_1)$", style="g-", data_style="k+")
plt.ylabel("$y - h_1(x_1) - h_2(x_1)$", fontsize=16)
plt.xlabel("$x_1$", fontsize=16)

plt.subplot(326)
plot_predictions([tree_reg1, tree_reg2, tree_reg3], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1) + h_3(x_1)$")
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$y$", fontsize=16, rotation=0);

On peut aussi utiliser la classe `GradientBoostingRegressor` de Scikit-Learn. Tout comme la classe RandomForestRegressor, elle possède des hyperparamètres pour contrôler la croissance des arbres de décision (par exemple, max_depth, min_samples_leaf, etc.), ainsi que des hyperparamètres pour contrôler la formation de l'ensemble, comme le nombre d'arbres (n_estimateurs). Le code suivant crée le même ensemble que le précédent : 

In [57]:
gbrt = sklearn.ensemble.GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=0.1, random_state=42)
gbrt.fit(X, y);

In [64]:
gbrt_slow = sklearn.ensemble.GradientBoostingRegressor(max_depth=2, n_estimators=50, learning_rate=0.1, random_state=42)
gbrt_slow.fit(X, y);

In [65]:
plt.figure(figsize=(11,4))

plt.subplot(121)
plot_predictions([gbrt], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="Ensemble predictions")
plt.title("learning_rate={}, n_estimators={}".format(gbrt.learning_rate, gbrt.n_estimators), fontsize=14)

plt.subplot(122)
plot_predictions([gbrt_slow], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("learning_rate={}, n_estimators={}".format(gbrt_slow.learning_rate, gbrt_slow.n_estimators), fontsize=14);

L'hyperparamètre `learning_rate` met à l'échelle la contribution de chaque arbre. Si vous le réglez sur une valeur faible, par exemple 0.1, vous aurez besoin de plus d'arbres dans l'ensemble pour fiter, mais les prédictions se généraliseront généralement mieux. Il s'agit d'une technique de régularisation appelée "shrinkage". 

***A vous:*** Trouvez de meilleurs paramètres $(2\heartsuit)$ (à la main, en testant). 


***A vous:*** Introduisez le learning_rate dans l'apprentissage "à la main" qu'on a fait précédemment. Il faut le mettre comme paramètre multiplicatif devant les prédictions des résidus. 

### early stopping

Afin de trouver le nombre optimal d'arbres, vous pouvez utiliser l'early stopping on arrête de rafiner dès que l'erreur de validation remonte. 

Ci-dessous on entraine un GBRT avec 120 arbres successif. Puis on revient en arrière pour n'en utiliser que le nombre optimal. 

On utilise la méthode `staged_predict()` : elle renvoie un itérateur sur les prédictions faites par l'ensemble à chaque étape d'entrainement.

In [ ]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=49)

gbrt = sklearn.ensemble.GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

errors = [np.mean( (y_val- y_pred)**2) for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators = np.argmin(errors)

gbrt_best = sklearn.ensemble.GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train);

In [ ]:
min_error = np.min(errors)

In [ ]:
plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(errors, "b.-")
plt.plot([bst_n_estimators, bst_n_estimators], [0, min_error], "k--")
plt.plot([0, 120], [min_error, min_error], "k--")
plt.plot(bst_n_estimators, min_error, "ko")
plt.text(bst_n_estimators, min_error*1.2, "Minimum", ha="center", fontsize=14)
plt.axis([0, 120, 0, 0.01])
plt.xlabel("Number of trees")
plt.title("Validation error", fontsize=14)

plt.subplot(122)
plot_predictions([gbrt_best], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("Best model (%d trees)" % bst_n_estimators, fontsize=14)

Il est également possible de mettre en œuvre l'early-stopping est d'arrêter l'entrainement dès que la validation commence à remonter. On définit pour cela un modèle avec l'option `warm_start=True`: quand on appelle la méthode `fit`, on repart de l'état du modèle obtenu par la précédente méthode `fit`.  


***A vous:*** En keras, est-ce que les modèles que l'on crée fonctionne comme des modèles sklearn avec l'option  `warm_start=True` ou `warm_start=False`?


Le code suivant arrête l'entrainement lorsque l'erreur de validation ne s'améliore pas pendant cinq itérations consécutives.


In [ ]:
gbrt = sklearn.ensemble.GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = np.mean ((y_val-y_pred)**2)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [ ]:
print(gbrt.n_estimators)

In [ ]:
print("Minimum validation MSE:", min_val_error)

### Subsample

La classe `GradientBoostingRegressor` a un hyperparamètre qui régle le sous-échantillonage du jeu d'entrainement. Avec `subsample = 0.25` chaque arbre est entrainé sur 25% des instances de `Train`, sélectionnées au hasard => plus de biais, moins de variance, modèle plus robuste et aussi plus rapide à entainer.  
Cette technique est appelée "Stochastic Gradient Boosting". 


## Exemple : Méthode d'ensemble sur les données le MNIST

Recette:
* Chargez les données du MNIST
*  divisez-les en `Train`, `Val`, `Test`
* entrainainez différents classificateurs
*  combinez-les en un ensemble
* Dégustez

In [ ]:
"le mnist avec les 70 000 images 28*28, mais c'est bien trop long"
#mnist = sklearn.datasets.fetch_mldata('MNIST original')
"on va utiliser le petit mnist"
mnist = sklearn.datasets.load_digits()
mnist.data.shape

Train/validation/test split

In [ ]:
nb_data=len(mnist.data)
X_train_val, X_test, y_train_val, y_test = sklearn.model_selection.train_test_split(
    mnist.data, mnist.target, test_size=nb_data//10, random_state=42)

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
    X_train_val, y_train_val, test_size=len(X_train_val)//9, random_state=42)

Entrainement individuel

In [ ]:
random_forest_clf = sklearn.ensemble.RandomForestClassifier(random_state=42)
extra_trees_clf = sklearn.ensemble.ExtraTreesClassifier(random_state=42)
mlp_clf = sklearn.neural_network.MLPClassifier(random_state=42)

mlp = multi-layer-perceptron = Réseau de neurone dense (full-connected)

In [ ]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("mlp_clf", mlp_clf)
]

In [ ]:
voting_clf = sklearn.ensemble.VotingClassifier(named_estimators)

On voit que par défaut, il s'agit d'un hard voter

In [ ]:
voting_clf.voting

On entraine les 3:

In [ ]:
voting_clf.fit(X_train, y_train);

In [ ]:
" the score of a classifier is the accuracy"
voting_clf.score(X_val, y_val)

On regarde les performances individuelles: 

In [ ]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

On enlève le pire: 

In [ ]:
del voting_clf.estimators_[0]

***Attention:*** l'attribut `estimators_` donne les modèles déjà entrainé, tandis que `estimators` donne la liste du modèle déclarés lorsque nous avons créé le `VotingClassifier`. Donc, ci-dessus, si nous faisions 

    del voting_clf.estimators[0]
    
la `forêt_aléatoire_clf` resterait active.

Maintenant, évaluons à nouveau le "VotingClassifier" :

In [ ]:
voting_clf.score(X_val, y_val)

Beaucoup mieux ! Le classement aléatoire des forêts nuisait à la performance. Essayons maintenant d'utiliser un classificateur de vote doux:

In [ ]:
voting_clf.voting = "soft"

In [ ]:
voting_clf.score(X_val, y_val)

Et sur l'ensemble Test ?

In [ ]:
voting_clf.score(X_test, y_test)

In [ ]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

Pas mal ! 